In [70]:
infile = "/data/Data/yntn/data_cleaned.log"
outfile = "/data/Data/yntn/data_out.csv"
emoji_path = '/data/Data/feed_data/new_cat'

In [71]:
with open(infile, "r") as f:
    lines = f.readlines()   

In [72]:
import json

# read file
with open('emoji2code.json', 'r') as myfile:
    data = myfile.read()

# parse file
obj = json.loads(data)
emoji = obj.keys()

In [73]:
from pathlib import Path

emoji_dict = {}

for file_dir in Path(emoji_path).glob('**/*.png'):
    if (str(file_dir.parent.stem) in emoji_dict):  
        emoji_dict[str(file_dir.parent.stem)].append(str(file_dir.stem))
    else:
        emoji_dict[str(file_dir.parent.stem)] = [str(file_dir.stem)]

In [74]:
import operator
import csv
import re, string
 
with open(outfile, "w") as f:
    writer = csv.writer(f)
    myData = [['text','label']]
    for line in lines:
        if ("feed at" in line):
            continue
        else:
            count = {}
            st = line
            for phrase in emoji:
                if (phrase in st):
                    count[obj[phrase]] = 0
                    while (phrase in st):
                        count[obj[phrase]] = count[obj[phrase]] + 1
                        st = st.replace(phrase,'',1)                    
            emoji_max = max(count.items(), key=operator.itemgetter(1))[0] #string
            st = st.lower()
            words = st.split()
            words = re.split(r'\W+', st)
            text = ''        
            text = ' '.join(word for word in words)
            if text.strip() == "" : 
                continue         
            for etype in emoji_dict.keys():
                if emoji_max in emoji_dict[etype]:
                    myData.append([text.strip(), etype.lower()])
                    break
                
    writer.writerows(myData)

In [75]:
import pandas as pd 
data = pd.read_csv('../data_out.csv') 
data

,text,label
0,ngon k c,suy ngẫm
1,sao e la i no i nv bô c hung dư lă m ha,khóc lớn
2,ok n cho cái hẹn đi,vui
3,thùy mị quá bạn ah,yêu
4,có đâu xé ib đi,ngạc nhiên
5,sao cũg đc hết,vui
6,e muốn ở tn mà,khóc lớn
7,e cũng ngủ ngon nhé,hôn
8,a ở lại đi,vui
9,lâu không gặp nhìn đẹp zai hẳn nha,cười gian
